## Introduction

The purpose of this workbook is to explore how to handle inconsistent data entry, meaning, handling values that are entered but not in a consistent manner. It will use sequence pattern matching to help isolate similar values so one can then clean them as needed.

It is based on Lesson 5 of the Kaggle Data Cleaning course
https://www.kaggle.com/alexisbcook/inconsistent-data-entry

## FuzzyWuzzy

The string matching FuzzyWuzzy library can give a score out of 100 to denote the similarity between strings. The higher the score, the higher the similarity. 

It was first developed to find and list event tickets off the internet. FuzzyWuzzy depends on the difflib library and Levenshtein distance to calculate the difference between patterns.

For more on approximate string matching, the FuzzyWuzzy and difflib libraries:
- https://en.wikipedia.org/wiki/Approximate_string_matching
- https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings
- https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
- https://docs.python.org/3/library/difflib.html

## Pakistan Intellectual Capital dataset

I found the dataset and info at:
https://www.kaggle.com/alexisbcook/pakistan-intellectual-capital

In [1]:
# pandas to read csv file
import pandas as pd

# see conclusion for more on this library
import chardet

# read in all our data
professors = pd.read_csv("pakistan_intellectual_capital.csv")

In [2]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [3]:
professors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 1142 non-null   int64  
 1   S#                                         1142 non-null   int64  
 2   Teacher Name                               1142 non-null   object 
 3   University Currently Teaching              1142 non-null   object 
 4   Department                                 1142 non-null   object 
 5   Province University Located                1142 non-null   object 
 6   Designation                                1123 non-null   object 
 7   Terminal Degree                            1138 non-null   object 
 8   Graduated from                             1142 non-null   object 
 9   Country                                    1142 non-null   object 
 10  Year                    

The focus will be on the non-null 'Country' column.

In [4]:
# 'Country' unique values
countries = professors['Country'].unique()

# sort them alphabetically
countries.sort()

# take a look
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

In [5]:
# 38 unique listed values in 'Country'
len(countries)

38

## Basic cleaning to improve consistency

### White spaces and uppercase letter

 As seen in 'countries', extra white spaces and/or uppercase letters result in some countries appearing twice. Using:
 - str.lower( ) will lowercase all strings, and 
 - str.strip( ) will remove extra spaces.

In [6]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()

# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

Another look at 'Country' unique values...

In [7]:
# 'Country' unique values
countries = professors['Country'].unique()

# sort them alphabetically
countries.sort()

# take a look
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

In [8]:
# from 38 -> 34: some duplicates were eliminated
len(countries)

34

### "urbana"

What is 'urbana'?

In [9]:
professors[professors['Country'] == 'urbana']

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
161,384,385,Sheikh Muhammad Qasim,Air University,Computer Science,Capital,Professor,PhD,University of Illinois,urbana,NaN,NaN,NaN


According to the info describing the dataset: https://www.kaggle.com/zusmani/pakistanintellectualcapitalcs

"The dataset contains list of computer science/IT professors from 89 different universities of Pakistan.

Variables:

The dataset contains Serial No, Teacher’s Name, University Currently Teaching, Department, Province University Located, Designation, Terminal Degree, Graduated from (university for professor), Country of graduation, Year, Area of Specialization/Research Interests, and some Other Information"

The University of Illinois is in 'usa', so I'll make the change for this professor.



In [10]:
professors.loc[professors['Country'] == 'urbana'] = 'usa'

In [11]:
countries = professors['Country'].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk', 'usa',
       'usofa'], dtype=object)

In [12]:
# from 34 -> 33
len(countries)

33

## Using FuzzyWuzzy

Basic cleaning has solved almost all data inconsistencies. But there are enough similarities in two pairs of countries to suggest they probably refer to one country, each:
- south korea: 'south korea' and 'southkorea'
- usa: 'usa'and 'usofa'

FuzzyWuzzy to the rescue!

In [13]:
# FuzzyWuzzy pattern matching tools
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Instead of just correcting by hand, FuzzyWuzzy can automate the cleaning.

In computer science, the formal term for fuzzy string search is called "Approximate string matching". This technique finds strings that approximately match the target pattern. The less changes needed to transform a string to the target string, the closer it is to the target.

In [14]:
# help(process.extract())

Looking at found at help(process.extract( )):
    
    
- query: the desired target string

- choices: an iterable or dictionary-like object containing choices to compare to the target string; in this case it is "countries", the sorted unique values in the 'Country' column.

- scorer: Optional function for scoring matches between the query and
        an individual processed choice. This should be a function
        of the form f(query, choice) -> int.
        By default, fuzz.WRatio() is used and expects both query and
        choice to be strings.        
I specified fuzz.token_sort_ratio


- limit = maximum elements to be returned; in this case, I specified the default 5


FuzzyWuzzy returns a ratio of the distance between two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to find the top five strings from our list of unique countries that have the closest distance to our target string.

process.extract( ) outputs a list of tuples of the found match and its score. 

### South Korea

In [15]:
# find the top 5 closest matches to "south korea"
matches_KR = process.extract(query="south korea",
                             choices=countries,
                             scorer=fuzz.token_sort_ratio,
                             limit=5)

# take a look at the matches for "south korea"
matches_KR

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('austria', 33)]

Of course, 'south korea' gets a score of 100, as it is exactly the same string specified in the 'query' argument. The string 'southkorea' has a score of 48, in terms of matching with the target. The other matches don't seem relevant.

To automate the change to the desired target, a function can be used.  

I modified the function provided in the Kaggle lesson to make it more generic to be able to use it for 'usa', an extra step that was not covered. I did this by adding the 'min_ratio' parameter. I also created a detailed docstring.

In [16]:
def replace_matches_in_column(df, column, string_to_match, min_ratio):
    '''
    A function that takes outputted strings and scores of process.extract()
    and changes them to the specified string_to_match,
    when the fuzz.token_sort_ratio is equal or higher than the specified min_ratio.
    
    Parameters:
    df: dataframe name
    column(str): specify the df column
    string_to_match(str): the desired target string to match
    min_ratio(int): the minimum score from process.extract scorer required to be affected by the change
    
    Returns:
    print statement "All done!" once the changes have been completed.
    
    '''
    
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 5 closest matches to our input string
    matches = process.extract(string_to_match,
                              strings,
                              limit=5,
                              scorer=fuzz.token_sort_ratio)

    # only get matches with a specified min_ratio
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

Let's see the function in action.

In [17]:
# replace close matches to "south korea" with "south korea"
# I chose min_ratio=47, because that was the min_ratio threshold that had best matches in matches_KR
replace_matches_in_column(df=professors,
                          column='Country',
                          string_to_match="south korea",
                          min_ratio=47)

All done!


In [18]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'usa', 'usofa'], dtype=object)

In [20]:
# from 33 -> 32
len(countries)

32

### United States

In [21]:
# find the top 5 closest matches to "usa"
matches_US = process.extract(query="usa",
                             choices=countries,
                             scorer=fuzz.token_sort_ratio,
                             limit=5)

# take a look at the matches for "usa"
matches_US

[('usa', 100),
 ('usofa', 75),
 ('austria', 60),
 ('australia', 50),
 ('spain', 50)]

Of course, 'usa' gets a score of 100, as it is exactly the same string specified in the 'query' argument. The string 'usofa' has a score of 75, in terms of matching with the target. The other matches don't seem relevant.

In [22]:
# replace close matches to "usa" with "usa"
# I chose min_ratio=75, because that was the min_ratio threshold that had best matches in matches_US
replace_matches_in_column(df=professors,
                          column='Country',
                          string_to_match="usa",
                          min_ratio=75)

All done!


In [23]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'usa'], dtype=object)

In [24]:
# from 32 -> 31 unique values
len(countries)

31

## Conclusion

### chardet
The lesson imports included import chardet. As far as I could determine, it wasn't needed in this example. However, I still decided to look it up and found some interesting pages:
- chardet documentation at https://chardet.readthedocs.io/en/latest/ 
- The research paper "A composite approach to language/encoding detection" which led to the Mozilla implementation at https://www-archive.mozilla.org/projects/intl/universalcharsetdetection



### 'choices' in process.extract( )

In this example, the unique values of the 'Country' column was used as the 'choices' argument in process.extract( ). Knowing that there are standard country code lists and dictionaries, means process.extract can become quite powerful as an efficient way to standardize inconsistent data entry for countries and many more values with standards strings already set.

I really enjoyed learning how to use FuzzyWuzzy.